In [1]:
import logging
logging.getLogger("tensorflow").setLevel(logging.DEBUG)

import tensorflow as tf
import numpy as np
print("TensorFlow version: ", tf.__version__)

TensorFlow version:  2.11.0


In [2]:
# Load MNIST dataset
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images.astype(np.float32) / 255.0
test_images = test_images.astype(np.float32) / 255.0

# Define the model architecture
model = tf.keras.Sequential([
  tf.keras.layers.InputLayer(input_shape=(28, 28)),
  tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
  tf.keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              metrics=['accuracy'])
model.fit(
  train_images,
  train_labels,
  epochs=5,
  validation_data=(test_images, test_labels)
)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3521 - accuracy: 0.8972 - val_loss: 0.2113 - val_accuracy: 0.9413
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1624 - accuracy: 0.9538 - val_loss: 0.1111 - val_accuracy: 0.9685
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0983 - accuracy: 0.9728 - val_loss: 0.0818 - val_accuracy: 0.9759
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0747 - accuracy: 0.9783 - val_loss: 0.0732 - val_accuracy: 0.9771
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0632 - accuracy: 0.9816 - val_loss: 0.0597 - val_accuracy: 0.9800


In [9]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(train_images).batch(1).take(100):
    yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8


In [10]:
tflite_model_quant = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\srico\AppData\Local\Temp\tmp3u_rxyjn\assets


INFO:tensorflow:Assets written to: C:\Users\srico\AppData\Local\Temp\tmp3u_rxyjn\assets
c:\Users\srico\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\lite\python\convert.py:765: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


In [15]:
with open('mnist_example.tflite', 'wb') as f:
    f.write(tflite_model_quant)

FileNotFoundError: [Errno 2] No such file or directory: 'models\\mnist_example.tflite'